# Matrix #1

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [2]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [3]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

def fortnight(day):
    if int(day) <15:
        return 1
    else:
        return 2

def activity(years):
    if int(years) <2013:
        return 2
    else:
        return 1

In [4]:
df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["jour (0)"]=df.apply(lambda row: monthCorrect(row['jour (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)
df["fortnight"]=df.apply(lambda row: fortnight(row["jour (0)"]), axis=1)
df["activity"]=df.apply(lambda row: fortnight(row["Annee (0)"]), axis=1)

#  

# Matrice 1 : Agrégation par année, mois, zone et type de bateaux

## Création de l'agrégation

Dans cette partie, notre but est de créér un dataframe agrégé par Année, mois, quinzaine, zones et type de bateaux.

On réalise un group by du dataframe original.

In [12]:
dfAgregateByArea=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","mois (0)","area_by_hand","fortnight","new_type"] ).size()}).reset_index()

In [13]:
print dfAgregateByArea.head(10)
print ""
print "-------------------------------------------------"
print "-------------------------------------------------"
print "La somme du nombre d'attaque total dans la zone :"
print dfAgregateByArea["count"].sum()

   Annee (0) mois (0) area_by_hand  fortnight         new_type  count
0       2008       02           AL          1  Vehicle Carrier      1
1       2008       02           AP          1            Other      1
2       2008       02            K          1           Tanker      1
3       2008       02            K          1              Tug      1
4       2008       03           AI          1            Other      1
5       2008       03           AP          1            Other      1
6       2008       03           AP          2            Other      1
7       2008       03            J          2           Tanker      1
8       2008       03            K          2  Vehicle Carrier      1
9       2008       03            M          2           Vessel      1

-------------------------------------------------
-------------------------------------------------
La somme du nombre d'attaque total dans la zone :
883


## Creation empty dataframe

On créé un dataframe avec la colonne 'Count' =0. On créé ainsi un dataframe avec toutes les possibilités d'année, mois, zone, quinzaine et type de bateaux

In [14]:
uniqueArea=sorted(dfAgregateByArea["area_by_hand"].unique())
uniqueYears=dfAgregateByArea['Annee (0)'].unique()
uniqueTypeBoat=dfAgregateByArea['new_type'].unique()
uniqueMonth=np.linspace(1, 12, num=12)
uniqueBinary=np.linspace(1, 2, num=2)



ColYears=np.repeat(uniqueYears,len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueArea)*len(uniqueBinary))
ColMonth=np.tile(np.repeat(uniqueMonth,len(uniqueArea)*len(uniqueBinary)*len(uniqueTypeBoat)), len(uniqueYears))
ColArea=np.tile(uniqueArea,len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueBinary)*len(uniqueYears))
ColType=np.tile(np.repeat(uniqueTypeBoat,len(uniqueArea)*len(uniqueBinary)),len(uniqueMonth)*len(uniqueYears))
ColFortnight=np.tile(np.repeat(uniqueBinary, len(uniqueArea)),len(uniqueMonth)*len(uniqueTypeBoat)*len(uniqueYears))
ColCount=np.repeat(0, len(ColFortnight))


In [15]:
dic = {'Years': ColYears,'Month': ColMonth,'Fortnight': ColFortnight, 'Area': ColArea,'Type':ColType,'Count': ColCount}
dfFinalArea= pd.DataFrame(dic)
print dfFinalArea.head(10)

  Area  Count  Fortnight  Month             Type  Years
0    A      0          1      1  Vehicle Carrier   2008
1   AA      0          1      1  Vehicle Carrier   2008
2   AB      0          1      1  Vehicle Carrier   2008
3   AC      0          1      1  Vehicle Carrier   2008
4   AE      0          1      1  Vehicle Carrier   2008
5   AF      0          1      1  Vehicle Carrier   2008
6   AG      0          1      1  Vehicle Carrier   2008
7   AH      0          1      1  Vehicle Carrier   2008
8   AI      0          1      1  Vehicle Carrier   2008
9   AJ      0          1      1  Vehicle Carrier   2008


In [17]:
print "On obtient alors un dataframe de taille : "+str(len(dfFinalArea))

On obtient alors un dataframe de taille : 60480


## Remplissage dataframe

On remplit ce dataframe avec l'agrégation.

In [19]:
for index, row in dfAgregateByArea.iterrows():
    idx=dfFinalArea[(dfFinalArea['Years']==row['Annee (0)']) & (dfFinalArea['Area']==row['area_by_hand']) 
                    & (dfFinalArea['Type']==row['new_type'])& (dfFinalArea['Fortnight']==row['fortnight'])
                    & (dfFinalArea['Month']==int(row["mois (0)"]))]
    dfFinalArea.set_value(idx.index.values, "Count",row['count'])

In [21]:
def getActivity(Years):
    activity=1
    if Years<2013:
        activity=2     
    return activity

dfFinalArea["Activity"]=dfFinalArea.apply(lambda row: getActivity(row['Years']), axis=1)

In [22]:
dfFinalArea.to_csv("../data/aden_golfe_matrix_area_hand.csv",sep=';', encoding='utf-8', index=False)

# Matrice 2 : Agrégation par année, mois et type de bateaux

On créé un dataframe agrégé par année, par mois et par type de bateaux

In [5]:
dfAgregateTotalType=pd.DataFrame({'count' : df.groupby( [ "date","new_type"] ).size()}).reset_index()
dfAgregateTotalType["date"]=dfAgregateTotalType["date"]+"-01"
print dfAgregateTotalType.head(5)

         date         new_type  count
0  2008-02-01            Other      1
1  2008-02-01           Tanker      1
2  2008-02-01              Tug      1
3  2008-02-01  Vehicle Carrier      1
4  2008-03-01            Other      3


On créé un dataframe vide.

In [6]:
max_month=12
unique_years=sorted(df["Annee (0)"].unique())
unique_type=sorted(dfAgregateTotalType["new_type"].unique())
month_repeat=np.linspace(1, max_month, num=max_month)

colYears=np.repeat(unique_years, max_month)
colMonth=np.tile(month_repeat, len(unique_years))
                               
dic2 = {'Years': colYears, 'Month': colMonth}
dfGlobale = pd.DataFrame(dic2)

for i in range(0,len(unique_type)):
    nameColLat=str(unique_type[i])
    dfGlobale[nameColLat]=0


dfGlobale.head()
dfGlobale["Month"]=dfGlobale.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfGlobale["date"]=dfGlobale.apply(lambda row: str(row["Years"]) +"-"+ str(row["Month"])+"-01", axis=1)
dfGlobale = dfGlobale.drop('Month', 1)                            
dfGlobale = dfGlobale.drop('Years', 1)
print dfGlobale.head()

   Barge  Military  Offshore Exploitation  Other  Ship  Tanker  Tug  \
0      0         0                      0      0     0       0    0   
1      0         0                      0      0     0       0    0   
2      0         0                      0      0     0       0    0   
3      0         0                      0      0     0       0    0   
4      0         0                      0      0     0       0    0   

   Vehicle Carrier  Vessel        date  
0                0       0  2008-01-01  
1                0       0  2008-02-01  
2                0       0  2008-03-01  
3                0       0  2008-04-01  
4                0       0  2008-05-01  


## Remplissage dataframe

In [11]:
for index, row in dfAgregateTotalType.iterrows():
    idx=dfGlobale[(dfGlobale['date']==row['date'])]
    dfGlobale.set_value(idx.index.values, str(row["new_type"]),row['count'])
print dfGlobale[["date","Vehicle Carrier","Vessel","Other","Ship","Offshore Exploitation"]].head(10)

         date  Vehicle Carrier  Vessel  Other  Ship  Offshore Exploitation
0  2008-01-01                0       0      0     0                      0
1  2008-02-01                1       0      1     0                      0
2  2008-03-01                1       1      3     0                      0
3  2008-04-01                2       3      2     0                      0
4  2008-05-01                5       0      1     0                      0
5  2008-06-01                1       0      1     0                      0
6  2008-07-01                2       0      2     0                      0
7  2008-08-01                6       0      4     0                      0
8  2008-09-01               13       1      4     0                      0
9  2008-10-01                8       0      3     0                      0


In [8]:
dfGlobale.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)